**Have you asked yourself why am I going to use PCA?**

We know from the previous attempts that :
- Our model overfits.
- There are outliers.
- There is some dependancy in between some fo the feautres.

**Will PCA solve all of these problems?**

*No, PCA will reduce the dimensionality but it doesn't gurantee that it will solve all the issue with the model. The most important part is that PCA is a step in the right direction.*

#### Here we start from the point where PCA can be applied 


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as se
from sklearn.model_selection import train_test_split


%matplotlib inline

data = pd.read_csv('training.csv')
data = data.drop('Unnamed: 0', axis=1)

data = data.fillna(data.mean())

# Separating train and test samples by 30 for the testing 

X = data [['RevolvingUtilizationOfUnsecuredLines', 'age',
          'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
          'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
          'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
          'NumberOfDependents']]
y = data [['SeriousDlqin2yrs']]

*I hope you know that there are multiple ways in which we can reduce the dimensions of our feature space. In general they fall in two categories:*

- Feature Elemination.
- Feature Extraction.

When we eleminate feautres we basically remove/drop/get rid of some features. It is a simple approach and easy to implement, but the drawback is that we gain no information from such method. If you drop them then you have exclude said features.

Extracting feautres however is creating new varibales in a way and order that remove dependency between our features. The new feautres are engineered in a way that by how good are they in predicting our dependant variable.

**Where does the deminsionality reduction come to play?"**

In a nut shell we re-produce our engineerd features (features extraction) and then select only the feautres that are import and drop the least importans ones. In fact our new feautres are a combinations of the old feautres, so even when we drop the least important engineered features we still preserve information from the old feautres.

**The following questions determine if you need to use PCA:**

- Do you want to reduce the number of variables, but aren’t able to identify variables to completely remove from consideration?
- Do you want to ensure your variables are independent of one another?
- Are you comfortable making your independent variables less interpretable?

If you answered all **yes** then PCA is the go to, if you answered the last question with **no** then maybe you should look at something else.

## How does PCA work?

- **Calculate a matrix that summarizes the relations between our feautres.**
- **Separate this matrix into two components, direction and magnitude.**
- **Transform original data to align with the important directions**

[Visual Explanation for PCA](http://setosa.io/ev/principal-component-analysis/).

Here we will follow the steps:
    - Create a pipline versioning two copies of the dataset (scaled and unscaled).
    - Use a classifier to be trained in each sample.
    - Plot the results of each classifier.

In [5]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline
from sklearn import linear_model


RANDOM_STATE = 42
FIG_SIZE = (10, 7)



# Make a train/test split using 30% test size
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30,
                                                    random_state=RANDOM_STATE)

# Fit to data and predict using pipelined GNB and PCA.
unscaled_clf = make_pipeline(PCA(n_components=2), GaussianNB())
unscaled_clf.fit(X_train, y_train)
pred_test = unscaled_clf.predict(X_test)

# Fit to data and predict using pipelined scaling, GNB and PCA.
std_clf = make_pipeline(StandardScaler(), PCA(n_components=2), GaussianNB())
std_clf.fit(X_train, y_train)
pred_test_std = std_clf.predict(X_test)

# Show prediction accuracies in scaled and unscaled data.
print('\nPrediction accuracy for the normal test dataset with PCA')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test)))

print('\nPrediction accuracy for the standardized test dataset with PCA')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, pred_test_std)))

# Extract PCA from pipeline
pca = unscaled_clf.named_steps['pca']
pca_std = std_clf.named_steps['pca']

# Show first principal componenets
print('\nPC 1 without scaling:\n', pca.components_[0])
print('\nPC 1 with scaling:\n', pca_std.components_[0])

/Users/faris/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/faris/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Prediction accuracy for the normal test dataset with PCA
93.38%


Prediction accuracy for the standardized test dataset with PCA
93.32%


PC 1 without scaling:
 [ 1.23083353e-04  3.31421345e-05 -2.31908924e-06 -7.24281787e-04
  9.99999729e-01  2.73775781e-05 -2.78386658e-06  8.34464585e-06
 -2.43504700e-06  4.24402195e-06]

PC 1 with scaling:
 [-5.32587021e-04 -5.88646804e-02  5.72239188e-01 -1.39046038e-02
 -1.39087397e-02 -7.81773124e-02  5.74185996e-01 -5.38244098e-02
  5.74393171e-01 -8.02215652e-03]
